# File for testing the initial generation pipeline for political ads

In [1]:
import Utils.Chat_GPT_Funcs as GPT

# Specify Inputs

In [5]:
ad_length_time = 30
ad_length_words = 75

candidate = "Mr. Man"

party = "Republican"
position = "Congress"

constituency = "South Georgia"

talking_points = "Cracking down on illegal immigration, building Trump's wall"

target_audiences = ["Farmers and agricultural workers",
                    "Small business owners and entrepreneurs",
                    "Conservative-leaning voters"]

# Setup and Generate the first draft of the ad

In [11]:
pol_consultant_role = GPT.open_file("Prompts/political_consultant_role_v1.txt")

tags = {
    '<<CANDIDATE>>': candidate,
    '<<POSITION>>': position,
    '<<TALKING_POINTS>>': talking_points,
    '<<CONSTITUENCY>>': constituency,
    '<<TARGET_AUDIENCE>>': target_audiences[0],
}

ad = GPT.generalized_gpt_prompt("Prompts/ad_generation_v1.txt", tags, role = pol_consultant_role)
print(ad)

Scene 1:
Dialogue: Friends in South Georgia, I'm Mr. Man, and I'm RUNNING for Congress - to PROTECT our farms and communities.
Cut Scene: Mr. Man standing in front of a farm, smiling and waving at the camera.

Scene 2:
Dialogue: I'll CRACK DOWN on illegal immigration - ensuring our farmers have FAIR, LEGAL labor.
Cut Scene: Mr. Man shaking hands with a farmer in a field.

Scene 3:
Dialogue: And I'll FIGHT for Trump's WALL - to keep our borders SECURE and our agriculture THRIVING.
Cut Scene: Mr. Man standing near a border fence with a determined expression.

Scene 4:
Dialogue: With your VOTE, we'll grow TOGETHER, for a STRONGER South Georgia!
Cut Scene: Mr. Man surrounded by a group of farmers and agricultural workers, all smiling and looking hopeful.


# Create the Target Personas

In [23]:
# Load the persona gen role
persona_gen_role = GPT.open_file("Prompts/persona_generator_v1.txt")

# Create the personas
persona_text = GPT.generalized_gpt_prompt("Prompts/persona_input.txt", tags, role = persona_gen_role)

In [25]:
personas = persona_text.split('\n\n')
for persona in personas:
    print(persona)
    print('\n')

Persona 1:
Demographics:
    - Name: John Thompson
    - Age: 45
    - Gender: Male
    - Nationality: American
    - Occupation: Farm owner
    - Description: John is a conservative farm owner who employs seasonal agricultural workers. He is concerned about the impact of illegal immigration on the country and the labor market.
    - Relevant information: John relies on a mix of local and immigrant workers to manage his farm, but he is concerned about the legality of some of his workers' immigration status.
    - Top Priorities:
        1. Ensuring that his farm has access to legal labor.
        2. Supporting policies that protect American jobs and control immigration.
    - Pain points:
        1. Difficulty finding reliable and legal seasonal workers.
        2. Concerns about the costs and effectiveness of building the border wall.


Persona 2:
Demographics:
    - Name: Mariana Sanchez
    - Age: 34
    - Gender: Female
    - Nationality: Mexican-American
    - Occupation: Agricult

# Have each persona rate the ad

In [27]:
ratings = []

for persona in personas:
    reviewer_role = GPT.open_file("Prompts/persona_rater_v1.txt").replace('<<PERSONA>>', persona)
    
    ad_review_in = GPT.open_file("Prompts/rate_ad_input.txt").replace('<<AD>>', ad)
    
    rating = GPT.chat_gpt(ad_review_in, temp=0.7, role = reviewer_role)
    print(rating)
    print('\n')
    
    ratings.append(rating)

Overall Rating: 6/10

Message Clarity: 6/10
The ad communicates the main message of Mr. Man running for Congress to protect farms and communities. It proposes that cracking down on illegal immigration and supporting Trump's wall will secure the borders and help the agriculture industry. However, the ad could have been clearer in explaining how these policies will directly benefit farmers and the labor market.

Visual Appeal: 7/10
The ad's visual elements are good, with Mr. Man appearing in different settings related to farming and border security. The color scheme and imagery are appropriate for the subject matter. However, it could have been more dynamic and engaging by incorporating more varied visuals or graphics.

Emotional Resonance: 5/10
The ad attempts to evoke feelings of hope and determination, particularly in the final scene with Mr. Man surrounded by farmers and agricultural workers. However, the emotional resonance is not very strong, as the ad lacks strong storytelling ele

# Update the comercial to encorporate the feedback

In [38]:
# Merge the reviews together
merged_string = ""
for i, review in enumerate(ratings, start=1):
    if i > 1:
        merged_string += " "
    merged_string += f"\n\nReview {i}: \n{review}"
    
# Load the ad editor
ad_editor_role = GPT.open_file("Prompts/ad_editor_v1.txt")

# Load the editor in prompt
edit_ad_in = GPT.open_file("Prompts/ad_edit_in.txt")
edit_ad_in = edit_ad_in.replace('<<AD>>', ad)
edit_ad_in = edit_ad_in.replace('<<FEEDBACK>>', merged_string)

updated_ad = GPT.chat_gpt(edit_ad_in, temp=0.7, role=ad_editor_role)
print(updated_ad)

Scene 1:
Dialogue: South Georgia friends, I'm Mr. Man - FIGHTING for Congress to PROTECT our farms and communities.
Cut Scene: Mr. Man standing in front of a farm, smiling and waving at the camera.

Scene 2:
Dialogue: I'll create FAIR labor policies - SUPPORTING our farmers and RESPECTING all families.
Cut Scene: Mr. Man shaking hands with a diverse group of farmers in a field, showcasing unity.

Scene 3:
Dialogue: I'll work for SMART border security - ensuring a THRIVING agricultural economy for ALL.
Cut Scene: Mr. Man examining farming equipment with farmers, indicating knowledge and understanding.

Scene 4:
Dialogue: With your VOTE - we'll GROW together for a STRONGER, UNITED South Georgia!
Cut Scene: Mr. Man surrounded by a diverse group of farmers and agricultural workers, all smiling and looking hopeful.


# Rerate the ad

In [39]:
new_ratings = []

for persona in personas:
    reviewer_role = GPT.open_file("Prompts/persona_rater_v1.txt").replace('<<PERSONA>>', persona)
    
    ad_review_in = GPT.open_file("Prompts/rate_ad_input.txt").replace('<<AD>>', updated_ad)
    
    rating = GPT.chat_gpt(ad_review_in, temp=0.7, role = reviewer_role)
    print(rating)
    print('\n')
    
    new_ratings.append(rating)

Overall Rating: 7/10

Message Clarity: 7/10
The ad does a decent job of communicating its main message, which seems to be about protecting farms, creating fair labor policies, and supporting smart border security. However, it could be more specific about the policies it intends to implement and how they relate to the target audience's concerns, such as the legality of seasonal workers and controlling immigration.

Visual Appeal: 8/10
The visual elements of the ad are appealing, with scenes of a farm, farmers, and agricultural workers. The color schemes and imagery are appropriate for the intended message and should capture the viewer's attention. However, the ad could benefit from more dynamic visuals or transitions between scenes to make it even more engaging.

Emotional Resonance: 7/10
The ad evokes emotions such as unity, hope, and optimism through its imagery and dialogue. However, it could be more effective in addressing the pain points of the target audience (difficulty finding r

In [40]:
class Scene:
    def __init__(self, scene_number, dialogue="", cut_scene=""):
        self._scene_number = scene_number
        self._dialogue = dialogue
        self._cut_scene = cut_scene

    def get_scene_number(self):
        return self._scene_number

    def set_scene_number(self, scene_number):
        self._scene_number = scene_number

    def get_dialogue(self):
        return self._dialogue

    def set_dialogue(self, dialogue):
        self._dialogue = dialogue

    def get_cut_scene(self):
        return self._cut_scene

    def set_cut_scene(self, cut_scene):
        self._cut_scene = cut_scene

    def __str__(self):
        return f"Scene {self._scene_number}:\nDialogue: {self._dialogue}\nCut Scene: {self._cut_scene}"


In [41]:
def make_scenes(scene_text):
    scene_texts = scene_text.split('\n\n')
    scenes = []
    for i, text in enumerate(scene_texts):
        dialogue_start_index = text.find("Dialogue: ")
        cut_scene_start_index = text.find("Cut Scene: ")
        if dialogue_start_index == -1 or cut_scene_start_index == -1:
            continue
        dialogue = text[dialogue_start_index + len("Dialogue: "):cut_scene_start_index].strip()
        cut_scene = text[cut_scene_start_index + len("Cut Scene: "):].strip()
        scene = Scene(i+1, dialogue, cut_scene)
        scenes.append(scene)
    return scenes


In [42]:
scenes = make_scenes(updated_ad)

In [43]:
for s in scenes:
    print(s)
    print()

Scene 1:
Dialogue: South Georgia friends, I'm Mr. Man - FIGHTING for Congress to PROTECT our farms and communities.
Cut Scene: Mr. Man standing in front of a farm, smiling and waving at the camera.

Scene 2:
Dialogue: I'll create FAIR labor policies - SUPPORTING our farmers and RESPECTING all families.
Cut Scene: Mr. Man shaking hands with a diverse group of farmers in a field, showcasing unity.

Scene 3:
Dialogue: I'll work for SMART border security - ensuring a THRIVING agricultural economy for ALL.
Cut Scene: Mr. Man examining farming equipment with farmers, indicating knowledge and understanding.

Scene 4:
Dialogue: With your VOTE - we'll GROW together for a STRONGER, UNITED South Georgia!
Cut Scene: Mr. Man surrounded by a diverse group of farmers and agricultural workers, all smiling and looking hopeful.



In [45]:
from Utils.text_to_voice import create_voiceover

In [46]:
for s in scenes:
    create_voiceover(s.get_dialogue(), "scene" + str(s.get_scene_number()) +".mp3")

Text-to-speech conversion successful
Text-to-speech conversion successful
Text-to-speech conversion successful
Text-to-speech conversion successful
